---
skip_exec: true
skip_showdoc: true
---

# Speech to Text Datasets

> Speech to text datasets

In [ ]:
#| default_exp audio.datasets.stt

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import torch
from torch.utils.data import DataLoader, Dataset
import torchaudio
from lightning import LightningDataModule, LightningModule


from lhotse.dataset import BucketingSampler, OnTheFlyFeatures
from lhotse.dataset.collation import TokenCollater
from lhotse.recipes import download_librispeech, prepare_librispeech
from lhotse.dataset.vis import plot_batch
from lhotse import CutSet, RecordingSet, SupervisionSet, Fbank, FbankConfig

from matplotlib import pyplot as plt

from pathlib import Path
from pprint import pprint
from typing import List

from nimrod.text.tokenizers import CharTokenizer

## Lhotse-Based Dataset

In [ ]:
#| export

class STTDataset(Dataset):
    def __init__(self,
        tokenizer:TokenCollater, # text tokenizer
        num_mel_bins:int=80 # number of mel spectrogram bins
        ):
        self.extractor = OnTheFlyFeatures(Fbank(FbankConfig(num_mel_bins=num_mel_bins)))
        self.tokenizer = tokenizer

    def __getitem__(self, cuts: CutSet) -> dict:
        cuts = cuts.sort_by_duration()
        feats, feat_lens = self.extractor(cuts)
        tokens, token_lens = self.tokenizer(cuts)
        return {"feats_pad": feats, "ilens": feat_lens, "tokens_pad": tokens}



## LibriSpeech DataModule

In [ ]:
#| export
class LibriSpeechDataModule(LightningDataModule):
    def __init__(self,
        target_dir="../data/en", # where data will be saved / retrieved
        dataset_parts="mini_librispeech", # either full librispeech or mini subset
        output_dir="../recipes/stt/librispeech/data", # where to save manifest
        num_jobs=1 # num_jobs depending on number of cpus available
    ):
        super().__init__()
        self.save_hyperparameters(logger=False)
        self.libri = {}

    def prepare_data(self,) -> None:
        download_librispeech(target_dir=self.hparams.target_dir, dataset_parts=self.hparams.dataset_parts)

    def setup(self, stage = None):
        self.libri = prepare_librispeech(Path(self.hparams.target_dir) / "LibriSpeech", dataset_parts=self.hparams.dataset_parts, output_dir=self.hparams.output_dir, num_jobs=self.hparams.num_jobs)
        if stage == "fit" or stage == None:
            self.cuts_train = CutSet.from_manifests(**self.libri["train-clean-5"])
            self.cuts_test = CutSet.from_manifests(**self.libri["dev-clean-2"])
            self.tokenizer = TokenCollater(self.cuts_train)
            self.tokenizer(self.cuts_test.subset(first=2))
            self.tokenizer.inverse(*self.tokenizer(self.cuts_test.subset(first=2)))
        if stage == "test":
            self.cuts_test = CutSet.from_manifests(**self.libri["dev-clean-2"])
            self.tokenizer = TokenCollater(self.cuts_test)
            self.tokenizer(self.cuts_test.subset(first=2))
            self.tokenizer.inverse(*self.tokenizer(self.cuts_test.subset(first=2)))

    def train_dataloader(self):
        train_sampler = BucketingSampler(self.cuts_train, max_duration=300, shuffle=True) #, bucket_method="equal_duration")
        return DataLoader(STTDataset(self.tokenizer), sampler=train_sampler, batch_size=None, num_workers=2)

    def test_dataloader(self):
        test_sampler = BucketingSampler(self.cuts_test, max_duration=400, shuffle=False) #, bucket_method="equal_duration")
        return DataLoader(STTDataset(self.tokenizer), sampler=test_sampler, batch_size=None, num_workers=2)

    @property
    def model_kwargs(self):
        return {
            "odim": len(self.tokenizer.idx2token),
        }

## Usage

In [ ]:
dm = LibriSpeechDataModule(
    target_dir="../data/en", 
    dataset_parts="mini_librispeech",
    output_dir="../data/en/LibriSpeech",
    num_jobs=1
)

In [ ]:
# skip this at export time to not waste time
# download
dm.prepare_data()

In [ ]:
# libri = prepare_librispeech("../data/en/LibriSpeech", dataset_parts='mini_librispeech')

In [ ]:
#| notest
#! rm ../data/en/LibriSpeech/*.gz

In [ ]:
#| notest
# dm.setup(stage='test')
# dm.cuts_test

In [ ]:
recs = RecordingSet.from_file("../data/en/LibriSpeech/librispeech_recordings_dev-clean-2.jsonl.gz")
sup = SupervisionSet("../data/en/LibriSpeech/librispeech_supervisions_dev-clean-2.jsonl.gz")
print(len(recs),len(sup))

1089 68


In [ ]:
#| notest
# test_dl = dm.test_dataloader()
# b = next(iter(test_dl))
# print(b["feats_pad"].shape, b["tokens_pad"].shape, b["ilens"].shape)
# plt.imshow(b["feats_pad"][0].transpose(0,1), origin='lower')

# dm.tokenizer.idx2token(b["tokens_pad"][0])
# dm.tokenizer.inverse(b["tokens_pad"][0], b["ilens"][0])

In [ ]:
#| notest
# print(dm.cuts_test)
# cut = dm.cuts_test[0]
# # pprint(cut.to_dict())
# cut.plot_audio()

## MiniLibriSpeech Dataset

In [ ]:
class MiniLibriSpeechDataset(Dataset):
    def __init__(
        self,
        root:str,
        subset:str,
        audio_transform:torchaudio.transforms = None,
        text_transform = None
        ):

        self.data = [] # audiopath, text
        
        self.audio_transform = audio_transform
        self.text_transform = text_transform

        path = Path(root) / subset
        for speaker_dir in path.iterdir():
            if not speaker_dir.is_dir():
                continue
            for chapter_dir in speaker_dir.iterdir():
                if not chapter_dir.is_dir():
                    continue
                trn_path = chapter_dir / f"{speaker_dir.name}-{chapter_dir.name}.trans.txt"
                if not trn_path.exists():
                    continue
                with open(trn_path, 'r') as f:
                    lines = f.readlines()

                with open(trn_path, 'r') as f:
                    lines = f.readlines()
                transcripts = {
                    line.split()[0]: ' '.join(line.strip().split()[1:])
                    for line in lines
                }

                for utt_id, text in transcripts.items():
                    audio_path = chapter_dir / f"{utt_id}.flac"
                    if audio_path.exists():
                        self.data.append((audio_path, text))
    
    def __getitem__(self, idx:int):
        audio_path, trn =  self.data[idx]
        waveform, sr = torchaudio.load(audio_path)
        if self.audio_transform:
            feats = self.audio_transform(waveform)
        else:
            feats = waveform
        
        if self.text_transform:
            label = self.text_transform(trn.lower())
        else:
            label = trn.lower()
        
        return feats.squeeze(0), label

    def __len__(self):
        return len(self.data)


## Audio & text transforms

In [ ]:
ds = MiniLibriSpeechDataset('../data/en/LibriSpeech', 'dev-clean-2')
print(len(ds), ds[0][0].shape, ds[0][1])

mfcc_transform = torchaudio.transforms.MFCC(sample_rate=16000, n_mfcc=13)
ds = MiniLibriSpeechDataset(
    "../data/en/LibriSpeech", "dev-clean-2",
    audio_transform=mfcc_transform
)
print(ds[0][0].shape, ds[0][1])

1089 torch.Size([186560]) if the reader will excuse me i will say nothing of my antecedents nor of the circumstances which led me to leave my native country the narrative would be tedious to him and painful to myself
torch.Size([13, 933]) if the reader will excuse me i will say nothing of my antecedents nor of the circumstances which led me to leave my native country the narrative would be tedious to him and painful to myself


In [ ]:
import string
vocab = list(string.ascii_lowercase + string.digits) + [' ', '.', ',', '?', '!', '-', "'"]
tok = CharTokenizer(vocab)
encoded = tok.encode('hello!')
decoded = tok.decode(encoded)
print(decoded)

ds = MiniLibriSpeechDataset(
    "../data/en/LibriSpeech", "dev-clean-2",
    audio_transform=mfcc_transform,
    text_transform=tok.encode
)
# print(ds[0][0].shape, ds[0][1])
print(ds[0][0].shape, tok.decode(ds[0][1]))

hello!
torch.Size([13, 933]) if the reader will excuse me i will say nothing of my antecedents nor of the circumstances which led me to leave my native country the narrative would be tedious to him and painful to myself


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()